In [137]:
import pandas as pd

In [146]:
df = pd.read_csv('data/nyc_bike_accidents.csv')
pd.options.display.max_columns = 50

In [147]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])


df['YEAR'] = df['CRASH DATE'].dt.strftime('%Y')
df

,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,YEAR
0,27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"(40.687534, -73.9775)",FULTON STREET,SAINT FELIX STREET,NaN,1.0,0.0,1,0,Unspecified,Unspecified,NaN,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN,2021
1,31,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"(40.784615, -73.953964)",EAST 93 STREET,NaN,NaN,1.0,0.0,1,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4486581,Van,Bike,NaN,NaN,NaN,2021
2,51,2022-04-24,15:35,MANHATTAN,10019.0,40.767242,-73.986206,"(40.767242, -73.986206)",WEST 56 STREET,9 AVENUE,NaN,1.0,0.0,1,0,View Obstructed/Limited,Unspecified,NaN,NaN,NaN,4521853,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,2022
3,66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,1.0,0.0,1,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN,2021
4,72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"(40.640835, -73.98967)",12 AVENUE,41 STREET,NaN,1.0,0.0,1,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49505,1964049,2023-01-24,17:35,BROOKLYN,11219.0,40.636566,-73.997696,"(40.636566, -73.997696)",51 STREET,11 AVENUE,NaN,1.0,0.0,1,0,Unspecified,NaN,NaN,NaN,NaN,4600342,Bike,NaN,NaN,NaN,NaN,2023
49506,1964058,2023-01-24,15:57,BROOKLYN,11215.0,40.667250,-73.994550,"(40.66725, -73.99455)",3 AVENUE,16 STREET,NaN,1.0,0.0,1,0,Unsafe Lane Changing,Unspecified,NaN,NaN,NaN,4600353,Sedan,Bike,NaN,NaN,NaN,2023
49507,1964064,2023-01-24,14:10,BRONX,10454.0,40.810726,-73.924650,"(40.810726, -73.92465)",EAST 139 STREET,ALEXANDER AVENUE,NaN,1.0,0.0,1,0,Unsafe Speed,Traffic Control Disregarded,NaN,NaN,NaN,4600464,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,2023
49508,1964105,2023-01-24,15:00,BROOKLYN,11220.0,40.640263,-74.019035,"(40.640263, -74.019035)",4 AVENUE,61 STREET,NaN,1.0,0.0,1,0,Unspecified,Unspecified,NaN,NaN,NaN,4600585,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,2023


In [150]:
bike_df = df[df['VEHICLE TYPE CODE 1'] != 'Bike']
bike_df = bike_df.groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().sort_values('NUMBER OF PERSONS INJURED', ascending=False).reset_index()
bike_df = bike_df[['CONTRIBUTING FACTOR VEHICLE 1', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED']]

# Drop the first row, because that is the cases where the reason was unspecified.
bike_df.drop(index=0, inplace=True)

# Get top 10 reasons
bike_df = bike_df.head(10)
bike_df

/var/folders/hx/m3n3wwr91yg_snnv68ynqsp80000gn/T/ipykernel_34374/2885368575.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bike_df = bike_df.groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().sort_values('NUMBER OF PERSONS INJURED', ascending=False).reset_index()


,CONTRIBUTING FACTOR VEHICLE 1,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED
1,Driver Inattention/Distraction,9964,28
2,Failure to Yield Right-of-Way,4735,9
3,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,2498,23
4,Traffic Control Disregarded,1692,13
5,Passenger Distraction,1603,8
6,Passing or Lane Usage Improper,1108,4
7,Other Vehicular,845,5
8,Turning Improperly,675,0
9,View Obstructed/Limited,566,1
10,Following Too Closely,543,1


In [ ]:
bike_df.to_csv('data/nyc_accidents_reason.csv')

In [142]:
bike_df_2022 = df[df['YEAR'] == '2022']

bike_df_2022 = bike_df_2022[bike_df_2022['VEHICLE TYPE CODE 1'] != 'Bike']
bike_df_2022 = bike_df_2022.groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().sort_values('NUMBER OF PERSONS INJURED', ascending=False).reset_index()
bike_df_2022 = bike_df_2022[['CONTRIBUTING FACTOR VEHICLE 1', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED']]
bike_df_2022

bike_df_2022.to_csv('data/nyc_accidents_reason_2022.csv')

/var/folders/hx/m3n3wwr91yg_snnv68ynqsp80000gn/T/ipykernel_34374/3142008372.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bike_df_2022 = bike_df_2022.groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().sort_values('NUMBER OF PERSONS INJURED', ascending=False).reset_index()


In [ ]:
df.groupby('YEAR').sum('NUMBER OF PERSONS KILLED')

,Unnamed: 0,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,COLLISION_ID
YEAR,,,,,,,,,
2012,4141793277,21810871.0,82042.521399,-148985.590667,2249.0,6.0,2210,6,346956363
2013,6971545221,40091890.0,151193.640470,-274536.520564,4120.0,11.0,4075,11,601651970
2014,6029479174,39608844.0,149907.740100,-272167.003660,4053.0,20.0,4000,20,4200328169
2015,5516546701,41577236.0,157682.844458,-286286.717771,4338.0,15.0,4281,15,13940478127
2016,5309160168,39324217.0,173723.351518,-315427.251408,5082.0,19.0,4975,18,17357233919
2017,4109693436,36155840.0,185856.188187,-337464.337220,4978.0,34.0,4889,27,18082238102
2018,2876710369,36349014.0,180279.764749,-327330.195093,4845.0,10.0,4725,10,18516588605
2019,1939985431,38538902.0,191075.813372,-346917.387261,5073.0,31.0,4986,31,20702042036
2020,1236231315,43037073.0,210935.405135,-382899.818754,5687.0,30.0,5576,29,23768782814


In [ ]:
# Get only the data from 2022
df_2022 = df[df['YEAR'] == '2022']

# Get only the columns we need
df_2022 = df_2022[['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF CYCLIST INJURED', 'ON STREET NAME', 'CROSS STREET NAME']]

# And create new columns that will be used for visualization
df_2022['ACCIDENT TYPE'] = 0

df_2022.loc[df_2022['NUMBER OF CYCLIST INJURED'] > 0, 'ACCIDENT TYPE'] = 'Injured'
df_2022.loc[df_2022['NUMBER OF CYCLIST KILLED'] > 0, 'ACCIDENT TYPE'] = 'Fatal'

df_2022['INJURED OR DEATH'] = df_2022['NUMBER OF CYCLIST KILLED'] + df_2022['NUMBER OF CYCLIST INJURED']

df_2022.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)

In [ ]:
df_2022.to_csv('data/nyc_bike_crashes_2022.csv')

df_2022.groupby('ON STREET NAME').sum().sort_values('INJURED OR DEATH', ascending=False).head(50)

/var/folders/hx/m3n3wwr91yg_snnv68ynqsp80000gn/T/ipykernel_34374/259462352.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2022.groupby('ON STREET NAME').sum().sort_values('INJURED OR DEATH', ascending=False).head(50)


,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF CYCLIST KILLED,NUMBER OF CYCLIST INJURED,INJURED OR DEATH
ON STREET NAME,,,,,,
BROADWAY,574128.0,3546.749792,-6432.960127,0,90,90
4 AVENUE,659666.0,2846.366428,-5179.957104,0,71,71
3 AVENUE,375181.0,2119.210039,-3845.993052,0,54,54
BEDFORD AVENUE,246820.0,1911.458332,-3475.896218,0,48,48
2 AVENUE,363463.0,1711.774058,-3106.570754,1,43,44
5 AVENUE,282135.0,1750.510665,-3181.335502,0,43,43
1 AVENUE,200441.0,1385.812630,-2514.717171,0,37,37
8 AVENUE,260243.0,1385.647392,-2515.101474,0,34,34
7 AVENUE,187526.0,1222.081275,-2219.443542,0,30,30


In [ ]:
accidents = pd.read_csv('data/nyc_bike_accidents.csv')

bikers = pd.read_csv('data/nyc_bikerides_numbers.csv')

In [ ]:
accidents['CRASH DATE'] = pd.to_datetime(accidents['CRASH DATE'])

accidents['YEAR'] = accidents['CRASH DATE'].dt.strftime('%Y')

accidents_year = accidents.groupby('YEAR').sum('NUMBER OF PERSONS KILLED')
accidents_year = accidents_year[['NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED']]
accidents_year

,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED
YEAR,,
2012,2210,6
2013,4075,11
2014,4000,20
2015,4281,15
2016,4975,18
2017,4889,27
2018,4725,10
2019,4986,31
2020,5576,29


In [ ]:
accidents_year = accidents_year.reset_index()

accidents_year.rename(columns={"YEAR": "year", "NUMBER OF CYCLIST INJURED": "injured", "NUMBER OF CYCLIST KILLED": "killed"}, inplace=True)

accidents_year

,year,injured,killed
0,2012,2210,6
1,2013,4075,11
2,2014,4000,20
3,2015,4281,15
4,2016,4975,18
5,2017,4889,27
6,2018,4725,10
7,2019,4986,31
8,2020,5576,29
9,2021,4961,19


In [ ]:
bikers['year'] = bikers['Unnamed: 0']
bikers = bikers[['year', 'Total Daily Cycling Trips']]
bikers

,year,Total Daily Cycling Trips
0,2008,240000.0
1,2009,240000.0
2,2010,250000.0
3,2011,270000.0
4,2012,320000.0
5,2013,380000.0
6,2014,420000.0
7,2015,450000.0
8,2016,460000.0
9,2017,490000.0


In [ ]:
bikers.drop([13], inplace=True)
bikers

,year,Total Daily Cycling Trips
0,2008,240000.0
1,2009,240000.0
2,2010,250000.0
3,2011,270000.0
4,2012,320000.0
5,2013,380000.0
6,2014,420000.0
7,2015,450000.0
8,2016,460000.0
9,2017,490000.0


In [ ]:
bikers_accidents = pd.merge(accidents_year, bikers, on=['year'], how='left')

bikers_accidents['injury_rate'] = bikers_accidents['injured'] / bikers_accidents['Total Daily Cycling Trips']
bikers_accidents['fatality_rate'] = bikers_accidents['killed'] / bikers_accidents['Total Daily Cycling Trips']


bikers_accidents.rename({'Total Daily Cycling Trips': 'total_daily_bikerides'}, axis=1, inplace=True)

bikers_accidents.to_csv('data/accidents_development.csv')